***This notebook is meant to be run in Google Colab***

#  Training a GPT-2 Text-Generating Model w/ GPU

[GPT-2 info from OpenAI](https://https://openai.com/blog/better-language-models/)

[GPT-2 Simple - Github Repo](https://github.com/minimaxir/gpt-2-simple)

*As of Labs37*

The BERT model we've inherited from prior cohorts has been suffering from a major class imbalance.

The data they collected for the training of this NLP classification model were unlabelled, prompting the creation of [this](http://bw-labeler-dev.us-east-1.elasticbeanstalk.com/). Ranks 2, 3, & 4 have but a few hundred samples among them, while Ranks 0, 1, & 5 contain thousands. Extreme cases of police activity are plenty, ranging from their complete absence to their murdering of innocent civillians. Rare is the opportunity to rank 'moderate' examples of police misconduct.

To alleviate this imbalance while side-stepping an otherwise [mundane task](https://www.kdnuggets.com/2017/06/acquiring-quality-labeled-training-data.html), Labs37 has decided to generate these tweets themselves. And what better way to fulfill this task than to use one of the most robust, open source, text-generating models around?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from time import time
import pandas as pd
!pip install psycopg2
import psycopg2

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## GPU

Colab uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [3]:
!nvidia-smi

Fri Sep 17 03:55:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Don't let   `0  Tesla T4            Off` fool you. It's on. Colab, for whatever reason, doesn't display whether their GPUs are enabled or not with the `nvidia-smi` shell command. 

If you're wondering if it's even on at all, try running that shell command again after changing your runtime back to **CPU**. It won't appear, but it will with a **GPU** runtime.

It's a shame it'll never display  `0  Tesla T4            On`.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [4]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 198Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.41Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 326Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:56, 24.9Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 218Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.27Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.15Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colab VM, and to get the trained model *out* of Colab, is to route it through Google Drive *first*.

Running this cell (which will only work in Colab) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [5]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Gathering Our Text Data

The tweets we'll be using to train our GPT-2 model must come in the form a of .txt file.

To do that we'll read in a .CSV file containing 5k tweets and isolate them from all other columns contained within their dataset.

We also want to train GPT-2 on each rank, **separately**, so as to ensure each batch of texts it'll generate only contains that rank, ensuring a simple labelling and compilation of our synthetic tweets.

In [ ]:
def get_df(db_url) -> pd.DataFrame:
    '''
    Connects to our Blue Witness Data Labeler and retrieves manually labelled text before converting them all into a pandas dataframe.
    
    Parameters
    ----------
    db_url: psycopg2 database
    
    Returns
    -------
    df: pandas datafarme 
        Contains thousands of text with appropriate police (non-)violence labels
    '''
    conn = psycopg2.connect(db_url)
    curs = conn.cursor()
    curs.execute("SELECT * FROM training;")
    cols = [k[0] for k in curs.description]
    rows = curs.fetchall()
    df = pd.DataFrame(rows, columns=cols)
    curs.close()
    conn.close()
    return df

In [ ]:
# ALWAYS REMEMBER TO REMOVE THE PostgreSQL URL ASSIGNED TO THIS VARIABLE WHEN COMITTING TO OUR REPO
db_url = ""

In [ ]:
data_labeler_df = get_df(db_url)
data_labeler_df.head()

,id,tweets,labels
0,6,A New Mexico State Police officer killed two p...,5
1,7,I found some police badge stickers at work and...,0
2,8,"According to media reports, police said they a...",1
3,9,The male who has the knife is now running nort...,0
4,10,Changing the assignments of already existing c...,0


In [ ]:
rank2 = pd.DataFrame(data_labeler_df[data_labeler_df['labels'] == 2])
rank2 = rank2['tweets']
rank3 = pd.DataFrame(data_labeler_df[data_labeler_df['labels'] == 3])
rank3 = rank3['tweets']
rank4 = pd.DataFrame(data_labeler_df[data_labeler_df['labels'] == 4])
rank4 = rank4['tweets']

In [ ]:
rank2.head()

119    UPDATE: Vacaville Police K9 Punched in the Fac...
131    New York police department,made in Israel!! Wa...
175    Bank holiday brawl between police and pub punt...
187    5 Bangladeshi nationals arrested in Bengaluru ...
200    4 volunteers with Black Protest Legal Support ...
Name: tweets, dtype: object

# Exporting our Isolated Tweets

Now that we've rittled down our dataframe to the sole column we need, it's time to export them. Pandas lacks a `.to_txt` method but can still get the job done by specifying the **type**(*in this instance, just adding .txt at the end*) of the file being exported with `.to_csv`.

In [ ]:
rank2.to_csv('rank2tweets.txt', sep='\t')
rank3.to_csv('rank3tweets.txt', sep='\t')
rank4.to_csv('rank4tweets.txt', sep='\t')

## Uploading a Text File to be Trained to Colab

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [8]:
file_name = "Run_over_by_cops.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [11]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [12]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=1000,
              restore_from='fresh',
              run_name='run5',
              print_every=5,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 3512.82it/s]

dataset has 2412 tokens
Training...


[5 | 15.59] loss=1.57 avg=1.57
[10 | 23.17] loss=0.74 avg=1.16
[15 | 30.84] loss=0.57 avg=0.96
[20 | 38.65] loss=0.23 avg=0.77
[25 | 46.56] loss=0.11 avg=0.64
[30 | 54.61] loss=0.02 avg=0.53
[35 | 62.75] loss=0.04 avg=0.46
[40 | 71.00] loss=0.27 avg=0.43
[45 | 79.42] loss=0.14 avg=0.40
[50 | 88.12] loss=0.05 avg=0.36
[55 | 97.02] loss=0.01 avg=0.33
[60 | 106.10] loss=0.01 avg=0.30
[65 | 115.11] loss=0.02 avg=0.28
[70 | 123.90] loss=0.02 avg=0.26
[75 | 132.54] loss=0.01 avg=0.24
[80 | 141.13] loss=0.06 avg=0.23
[85 | 149.67] loss=0.02 avg=0.22
[90 | 158.19] loss=0.02 avg=0.20
[95 | 166.78] loss=0.01 avg=0.19
[100 | 175.43] loss=0.02 avg=0.18
[105 | 184.17] loss=0.02 avg=0.17
[110 | 192.97] loss=0.01 avg=0.17
[115 | 201.75] loss=0.02 avg=0.16
[120 | 210.49] loss=0.02 avg=0.15
[125 | 219.25] loss=0.02 avg=0.15
[130 | 227.96] loss=0.14 avg=0.15
[135 | 236.64] loss=0.07 avg=0.14
[140 | 245.32] loss=0.02 avg=0.14
[145 | 253.97] loss=0.01 avg=0.13
[150 | 262.64] loss=0.01 avg=0.13
[155 | 271.

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [13]:
gpt2.copy_checkpoint_to_gdrive(run_name='run5')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [7]:
gpt2.copy_checkpoint_from_gdrive(run_name='run5')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [6]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run5')

Loading checkpoint checkpoint/run5/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run5/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [8]:
gpt2.generate(sess, run_name='run5')

The body camera released by authorities show Saulters driving behind Patmon as he runs down a street. Saulters can then be seen turning his steering wheel to the right as the car strikes Patmon and he slides onto the windscreen then onto the road. "We got him," says Saulters as his partner handcuffs the suspect./5
The dramatic dashcam video of a Marana patrol officer ramming an armed robbery suspect with his squad car has gone viral./5
Video footage shows a pedestrian in Little Rock being hit by a police vehicle and injured while crossing the street. Traffic camera footage from the intersection of West 12th and Peyton Street shows a man walking across the road around 3:12 p.m on May 22. Little Rock Police Officer Rashaud Williams turned into the lane the man was crossing and struck him./5
A police officer in Washington state was caught on video ramming his marked SUV into a crowd and running over a pedestrian Saturday night. The disturbing incident happened in downtown Tacoma, where so

# Analysis of Generated Tweets

Some of the tweets it's generating are actually the exact same tweets its been trained on.

"*Villanueva’s garment factory co-workers called police for help with his epileptic seizure. They arrived and found him uncooperative and combative. Wrestled to the ground and handcuffed by four officers, witnesses described police shouting for him to speak English, and taking his foaming at the mouth as a sign of drug use. He died of ""mechanical asphyxia following physical restraint"". The four were charged with manslaughter.*"

I found this tweet with a Google Search, while the tweet below was original:

"*According to police, officers responded at about 8 p.m. to a report that a man was breaking up an apartment. ""He was covered with blood when the officers arrived,"" said sheriff’s Capt. Mike McCabe. Officers alleged that Custer confronted them, then went into apparent cardiac arrest as they tried to restrain him. Custer died at a hospital at about 9 p.m. Custer’s family said the cause of his death was positional asphyxiation.*"

Considering the realism of both tweets, both the regurgitated training tweets and authentically generated tweets are nearly indistiguishable.

## Possible Solution:
A `unique()` function will be needed to check for duplicates before compiling these tweets with our already established dataset.

---

You also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [9]:
gpt2.generate(sess,
              run_name='run5',
              temperature=0.7,
              nsamples=5,
              batch_size=5
              )

The first lady of a Texas Department of Public Safety (DPS) trooper collided with a pedestrian early Wednesday morning, according to the Austin Police Department (APD). The crash happened at around 3:45 a.m. Thursday just west of the Texas State Capitol building. The trooper was heading northbound on Colorado Street and as he was turning left on 13th Street, the trooper hit the pedestrian. DPS said the crash happened while the trooper was patrolling the area./5
An on-duty police officer driving his patrol car hit and killed a pedestrian early Wednesday morning, police said. Police released a statement about the fatality but did not immediately identify the man who died or the officer who hit him. Police said the victim is between 30 and 40 years old and will be identified by the Solano County Coroner’s Office./5
A pedestrian died early Thursday morning after being struck by a Washington County Sheriff's Office (DPS) Office. Prior to 2 a.m., Solano County Sheriff's Office (DPS) Office r

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [10]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      run_name='run5',
                      destination_path=gen_file,
                      temperature=1.0,
                      nsamples=60,
                      batch_size=5
                      )

In [11]:
# may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Etc

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1